In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn import preprocessing, model_selection, metrics
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import target_encoding as te
import gc

# Text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import FeatureUnion
from scipy.sparse import hstack, csr_matrix
from nltk.corpus import stopwords 
import time
import string

#Images
import zipfile

In [2]:
train_df = pd.read_csv("../data/train.csv", parse_dates=["activation_date"], nrows=1000)
test_df = pd.read_csv("../data/test.csv", parse_dates=["activation_date"], nrows=1000)
aggregated_features = pd.read_csv("../data/aggregated_features.csv")
#periods_train = pd.read_csv("../data/periods_train.csv", parse_dates=["activation_date", "date_from", "date_to"])#, nrows=1000)
#periods_test = pd.read_csv("../data/periods_test.csv", parse_dates=["activation_date", "date_from", "date_to"])#, nrows=1000)
#train_active = pd.read_csv("../data/train_active.csv", parse_dates=["activation_date"], nrows=1000)
#test_active = pd.read_csv("../data/test_active.csv", parse_dates=["activation_date"], nrows=1000)
trainindex = train_df.index
testindex = test_df.index
test_id = test_df["item_id"].values
print("Train file rows and columns are : ", train_df.shape)
print("Test file rows and columns are : ", test_df.shape)


Train file rows and columns are :  (1000, 18)
Test file rows and columns are :  (1000, 17)


In [3]:
train_y = train_df.deal_probability.copy()
train_df.drop("deal_probability",axis=1, inplace=True)
train_df['param_1_enc'] = train_df['param_1']
train_df['param_2_enc'] = train_df['param_2']
train_df['param_3_enc'] = train_df['param_3']
test_df['param_1_enc'] = test_df['param_1']
test_df['param_2_enc'] = test_df['param_2']
test_df['param_3_enc'] = test_df['param_3']

In [4]:
# Calculate concurrent offers
#def calc_conc(df):
    

In [5]:
# Target encode the categorical variables #
cat_vars = ["region", "city", "parent_category_name", "category_name", "user_type", "param_1_enc", "param_2_enc", "param_3_enc", "image_top_1"]
for col in cat_vars:
    train_df[col], test_df[col] = te.target_encode(train_df[col], test_df[col], train_y, min_samples_leaf=100, smoothing=10, noise_level=0.01)
    #lbl = preprocessing.LabelEncoder()
    #lbl.fit(list(train_df[col].values.astype('str')) + list(test_df[col].values.astype('str')))
    #train_df[col] = lbl.transform(list(train_df[col].values.astype('str')))
    #test_df[col] = lbl.transform(list(test_df[col].values.astype('str')))


In [6]:
# Combine train and test for preprocessing
df = pd.concat([train_df,test_df],axis=0)
del train_df, test_df
gc.collect()

df.head()

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,param_1_enc,param_2_enc,param_3_enc
0,b912c3c6a6ad,e00f8ff2eaf9,0.148480,0.147969,0.067718,0.146525,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2,2017-03-28,0.162373,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,0.145699,0.145496,0.144121,0.147788
1,2dac0150717d,39aeb48f0017,0.144797,0.148847,0.194463,0.147428,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,0.165137,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,0.147802,0.146015,0.148922,0.148767
2,ba83aefab5dc,91e2f88dd6e3,0.147397,0.147911,0.158137,0.145060,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9,2017-03-20,0.165067,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,0.144518,0.146191,0.146872,0.148012
3,02996f1dd2ea,bf5cccea572d,0.147520,0.147528,0.066452,0.149121,Автомобильные кресла,NaN,NaN,Автокресло,Продам кресло от0-25кг,2200.0,286,2017-03-25,0.114081,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,0.148656,0.149209,0.146096,0.147213
4,7c90be56d2ab,ef50846afc0b,0.146581,0.147077,0.148329,0.146068,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",Все вопросы по телефону.,40000.0,3,2017-03-16,0.165932,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,0.148689,0.149094,0.144565,0.146713


In [7]:
df = df.merge(aggregated_features, on=['user_id'], how='left')
del aggregated_features
gc.collect()
df.head()

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,...,activation_date,user_type,image,image_top_1,param_1_enc,param_2_enc,param_3_enc,avg_days_up_user,avg_times_up_user,n_user_items
0,b912c3c6a6ad,e00f8ff2eaf9,0.148480,0.147969,0.067718,0.146525,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),...,2017-03-28,0.162373,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,0.145699,0.145496,0.144121,0.147788,8.000000,2.000000,2
1,2dac0150717d,39aeb48f0017,0.144797,0.148847,0.194463,0.147428,Другое,NaN,NaN,Стойка для Одежды,...,2017-03-26,0.165137,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,0.147802,0.146015,0.148922,0.148767,NaN,NaN,1
2,ba83aefab5dc,91e2f88dd6e3,0.147397,0.147911,0.158137,0.145060,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,...,2017-03-20,0.165067,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,0.144518,0.146191,0.146872,0.148012,4.428571,1.142857,9
3,02996f1dd2ea,bf5cccea572d,0.147520,0.147528,0.066452,0.149121,Автомобильные кресла,NaN,NaN,Автокресло,...,2017-03-25,0.114081,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,0.148656,0.149209,0.146096,0.147213,16.714286,2.642857,32
4,7c90be56d2ab,ef50846afc0b,0.146581,0.147077,0.148329,0.146068,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",...,2017-03-16,0.165932,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,0.148689,0.149094,0.144565,0.146713,NaN,NaN,1


In [8]:
# Simple Feature Engineering

# Time Data
df["activation_weekday"] = df["activation_date"].dt.weekday
#df["activation_monthday"] = df["activation_date"].dt.day

# Price
## Replace Nan with mean in price
#categories = df.category_name.unique()
#region = df.region.unique()
#param1 = df.param_1.unique()
#
#
#df["price_new"] = df["price"].values
#
#for cat in categories:
#    for reg in region:
#        cur_df = df.loc[(df["category_name"] == cat)  & (df["region"] == reg)]["price_new"]
#        cur_df.fillna(np.nanmean(cur_df.values), inplace=True)
#
#
#df["price"] = pd.isna(df["price"])
df["price"] = np.log(df["price"]+1)
df["price"].fillna(0, inplace=True)
df["image_top_1"].fillna(0, inplace=True)
df['param_1'].fillna("", inplace = True)
df['param_2'].fillna("", inplace = True)
df['param_3'].fillna("", inplace = True)



#Drop Cols
cols_to_drop = ["item_id", "user_id", "activation_date", "image"]
df.drop(cols_to_drop, axis=1,inplace=True)

In [9]:
# Image Featues
df.head()

,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,user_type,image_top_1,param_1_enc,param_2_enc,param_3_enc,avg_days_up_user,avg_times_up_user,n_user_items,activation_weekday
0,0.148480,0.147969,0.067718,0.146525,Постельные принадлежности,,,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",5.993961,2,0.162373,0.145699,0.145496,0.144121,0.147788,8.000000,2.000000,2,1
1,0.144797,0.148847,0.194463,0.147428,Другое,,,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",8.006701,19,0.165137,0.147802,0.146015,0.148922,0.148767,NaN,NaN,1,6
2,0.147397,0.147911,0.158137,0.145060,"Видео, DVD и Blu-ray плееры",,,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",8.294300,9,0.165067,0.144518,0.146191,0.146872,0.148012,4.428571,1.142857,9,0
3,0.147520,0.147528,0.066452,0.149121,Автомобильные кресла,,,Автокресло,Продам кресло от0-25кг,7.696667,286,0.114081,0.148656,0.149209,0.146096,0.147213,16.714286,2.642857,32,5
4,0.146581,0.147077,0.148329,0.146068,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",Все вопросы по телефону.,10.596660,3,0.165932,0.148689,0.149094,0.144565,0.146713,NaN,NaN,1,3


In [10]:
# Text Features
df['text_feat'] = df.apply(lambda row: ' '.join([
    str(row['param_1']), 
    str(row['param_2']), 
    str(row['param_3'])]),axis=1) # Group Param Features
df.drop(['param_1', 'param_2', 'param_3'], axis = 1, inplace=True)

In [11]:
# Punctuation in description
df['desc_punc'] = df['description'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
# Uppercase characters in description
df['upper_desc'] =  df['description'].apply(lambda x: len([c for c in str(x) if c.isupper()]))
# Meta Text Features
print(df)
textfeats = ["description", "text_feat", "title"]
for cols in textfeats:
    df[cols] = df[cols].astype(str) 
    df[cols] = df[cols].astype(str).fillna('empty') # FILL NA
    df[cols] = df[cols].str.lower() # Lowercase all text, so that capitalized words dont get treated differently
    df[cols + '_num_chars'] = df[cols].apply(len) # Count number of Characters
    df[cols + '_num_words'] = df[cols].apply(lambda comment: len(comment.split())) # Count number of Words
    df[cols + '_num_unique_words'] = df[cols].apply(lambda comment: len(set(w for w in comment.split())))
    df[cols + '_words_vs_unique'] = df[cols+'_num_unique_words'] / df[cols+'_num_words'] * 100 # Count Unique Words
    
df['relative_upper_desc'] = df['upper_desc'] / df['description_num_chars'] * 100

        region      city  parent_category_name  category_name  \
0     0.148480  0.147969              0.067718       0.146525   
1     0.144797  0.148847              0.194463       0.147428   
2     0.147397  0.147911              0.158137       0.145060   
3     0.147520  0.147528              0.066452       0.149121   
4     0.146581  0.147077              0.148329       0.146068   
5     0.151362  0.150588              0.067876       0.149249   
6     0.148196  0.147974              0.194805       0.147347   
7     0.145094  0.147863              0.068310       0.039529   
8     0.146991  0.148308              0.067938       0.040046   
9     0.147105  0.149529              0.067433       0.051698   
10    0.148636  0.148115              0.152255       0.149798   
11    0.145112  0.148010              0.066393       0.039703   
12    0.147987  0.146305              0.068034       0.149798   
13    0.143965  0.148699              0.157295       0.146334   
14    0.147742  0.147577 

In [12]:
df

,region,city,parent_category_name,category_name,title,description,price,item_seq_number,user_type,image_top_1,...,description_words_vs_unique,text_feat_num_chars,text_feat_num_words,text_feat_num_unique_words,text_feat_words_vs_unique,title_num_chars,title_num_words,title_num_unique_words,title_words_vs_unique,relative_upper_desc
0,0.148480,0.147969,0.067718,0.146525,кокоби(кокон для сна),"кокон для сна малыша,пользовались меньше месяц...",5.993961,2,0.162373,0.145699,...,100.000000,27,2,2,100.000000,21,3,3,100.0,1.724138
1,0.144797,0.148847,0.194463,0.147428,стойка для одежды,"стойка для одежды, под вешалки. с бутика.",8.006701,19,0.165137,0.147802,...,100.000000,8,1,1,100.000000,17,3,3,100.0,4.878049
2,0.147397,0.147911,0.158137,0.145060,philips bluray,"в хорошем состоянии, домашний кинотеатр с blu ...",8.294300,9,0.165067,0.144518,...,100.000000,29,5,5,100.000000,14,2,2,100.0,6.060606
3,0.147520,0.147528,0.066452,0.149121,автокресло,продам кресло от0-25кг,7.696667,286,0.114081,0.148656,...,100.000000,22,2,2,100.000000,10,1,1,100.0,4.545455
4,0.146581,0.147077,0.148329,0.146068,"ваз 2110, 2003",все вопросы по телефону.,10.596660,3,0.165932,0.148689,...,100.000000,26,5,5,100.000000,14,3,3,100.0,4.166667
5,0.151362,0.150588,0.067876,0.149249,авто люлька,в хорошем состоянии,7.170888,9,0.166337,0.146966,...,100.000000,22,2,2,100.000000,11,2,2,100.0,5.263158
6,0.148196,0.147974,0.194805,0.147347,водонагреватель 100 литров нержавейка плоский,электро водонагреватель накопительный на 100 л...,9.305741,125,0.164748,0.146193,...,90.476190,20,3,3,100.000000,45,5,5,100.0,4.255319
7,0.145094,0.147863,0.068310,0.039529,бойфренды colins,бойфренды в хорошем состоянии.,6.216606,61,0.167777,0.148149,...,100.000000,24,4,4,100.000000,16,2,2,100.0,3.333333
8,0.146991,0.148308,0.067938,0.040046,платье,54 раз мер очень удобное,6.216606,85,0.162327,0.150029,...,100.000000,39,8,8,100.000000,6,1,1,100.0,0.000000
9,0.147105,0.149529,0.067433,0.051698,полу ботиночки замш натур.бамбини,по стельке 15.5см мерить приокский район. цвет...,5.993961,136,0.115888,0.148063,...,100.000000,20,4,4,100.000000,33,4,4,100.0,3.076923


In [ ]:
out_df = df.head(trainindex.shape[0])
out_df['deal_probability']= train_y
out_df.head()
out_df.to_csv('dataframe_text_features.csv')

In [ ]:
print("\n[TF-IDF] Term Frequency Inverse Document Frequency Stage")
russian_stop = set(stopwords.words('russian'))

tfidf_para = {
    "stop_words": russian_stop,
    "analyzer": 'word',
    "token_pattern": r'\w{1,}',
    "sublinear_tf": True,
    "dtype": np.float32,
    "norm": 'l2',
    #"min_df":5,
    #"max_df":.9,
    "smooth_idf":False
}
def get_col(col_name): return lambda x: x[col_name]
vectorizer = FeatureUnion([
        ('description',TfidfVectorizer(
            ngram_range=(1, 2),
            max_features=16000,
            **tfidf_para,
            preprocessor=get_col('description'))),
        ('text_feat',CountVectorizer(
            ngram_range=(1, 2),
            #max_features=7000,
            preprocessor=get_col('text_feat'))),
        ('title',TfidfVectorizer(
            ngram_range=(1, 2),
            **tfidf_para,
            #max_features=7000,
            preprocessor=get_col('title')))
    ])
    
start_vect=time.time()
vectorizer.fit(df.loc[trainindex,:].to_dict('records'))
ready_df = vectorizer.transform(df.to_dict('records'))
tfvocab = vectorizer.get_feature_names()
print("Vectorization Runtime: %0.2f Minutes"%((time.time() - start_vect)/60))

# Drop Text Cols
df.drop(textfeats, axis=1,inplace=True)

In [ ]:
train_X = hstack([csr_matrix(df.head(trainindex.shape[0]).values),ready_df[0:trainindex.shape[0]]]) # Sparse Matrix
test_X = hstack([csr_matrix(df.tail(testindex.shape[0]).values),ready_df[trainindex.shape[0]:]])
tfvocab = df.columns.tolist() + tfvocab
for shape in [train_X,test_X]:
    print("{} Rows and {} Cols".format(*shape.shape))
print("Feature Names Length: ",len(tfvocab))
del df
gc.collect();

In [ ]:
def run_lgb(train_X, train_y, val_X, val_y, test_X):
    params = {
        "objective" : "regression",
        "metric" : "rmse",
        "num_leaves" : 30,
        "learning_rate" : 0.1,
        "bagging_fraction" : 0.7,
        "feature_fraction" : 0.7,
        "bagging_frequency" : 5,
        "bagging_seed" : 2018,
        "verbosity" : -1
    }
    
    lgtrain = lgb.Dataset(train_X, label=train_y, feature_name=tfvocab)
    lgval = lgb.Dataset(val_X, label=val_y, feature_name=tfvocab)
    evals_result = {}
    model = lgb.train(params, lgtrain, 16000, valid_sets=[lgval], early_stopping_rounds=500, verbose_eval=20, evals_result=evals_result)
    
    pred_test_y = model.predict(test_X, num_iteration=model.best_iteration)
    return pred_test_y, model, evals_result

In [ ]:

# Splitting the data for model training#
X_train, X_val, y_train, y_val = train_test_split(train_X, train_y, test_size=0.2, random_state=42)


# Training the model #
pred_test, model, evals_result = run_lgb(X_train, y_train, X_val, y_val, test_X)

# Making a submission file #
pred_test[pred_test>1] = 1
pred_test[pred_test<0] = 0
sub_df = pd.DataFrame({"item_id":test_id})
sub_df["deal_probability"] = pred_test
sub_df.to_csv("tfidf_aggregated_features_lgb.csv", index=False)

In [ ]:
fig, ax = plt.subplots(figsize=(12,18))
lgb.plot_importance(model, max_num_features=50, height=0.8, ax=ax)
ax.grid(False)
plt.title("LightGBM - Feature Importance", fontsize=15)
plt.show()